In [1]:
import duckdb as db
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [3]:
df = pd.DataFrame(sql(
    """  
    SELECT * 
    FROM name_basics 
    WHERE primaryProfession LIKE '%actor%' AND primaryProfession LIKE '%actress%'
    """
))

In [4]:
df_tmdb = pd.DataFrame(sql(
    """  
    SELECT * 
    FROM tmdb
    """
))

In [5]:
#Isolement des acteurs/actrices
df_act = df[df['primaryProfession'].str.contains("actress")]
df_act

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0004192,Kenny Hotz,1967,\N,"actress,actor,writer","tt0121955,tt0384746,tt1007028,tt0127388"
1,nm0164577,Tordy Clark,\N,\N,"actor,writer,actress","tt24248198,tt8323668,tt12724306,tt6561576"
2,nm0177403,Darlene Cooke,\N,\N,"actor,actress,miscellaneous","tt5296406,tt0164184,tt6759380,tt7573024"
3,nm0312197,ShaynaAngel Monroe,\N,\N,"actress,actor","tt1798701,tt1615919,tt1630574,tt0173716"
4,nm0377840,Gloria Lynne Henry,\N,\N,"actor,stunts,actress","tt0110823,tt0118971,tt3627704,tt0083620"
...,...,...,...,...,...,...
782,nm9875454,Mabel Thomas,\N,\N,"actor,actress","tt27427973,tt30956852,tt19245296,tt26315810"
783,nm9890521,Kendall Rose Dath,\N,\N,"actress,actor",tt8334452
784,nm9894699,Mia Tharia,\N,\N,"actor,writer,actress","tt14371256,tt29315015,tt19799292,tt8581660"
785,nm9919463,Rory Anne Dahl,\N,\N,"actor,actress",tt12483708


In [6]:
#Transofrmation de knownForTitles en liste
df_act['knownForTitles'] = df_act['knownForTitles'].apply(lambda x : x.split(","))
#explode
df_act_expl = df_act.explode("knownForTitles")
df_act_expl

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0004192,Kenny Hotz,1967,\N,"actress,actor,writer",tt0121955
0,nm0004192,Kenny Hotz,1967,\N,"actress,actor,writer",tt0384746
0,nm0004192,Kenny Hotz,1967,\N,"actress,actor,writer",tt1007028
0,nm0004192,Kenny Hotz,1967,\N,"actress,actor,writer",tt0127388
1,nm0164577,Tordy Clark,\N,\N,"actor,writer,actress",tt24248198
...,...,...,...,...,...,...
784,nm9894699,Mia Tharia,\N,\N,"actor,writer,actress",tt29315015
784,nm9894699,Mia Tharia,\N,\N,"actor,writer,actress",tt19799292
784,nm9894699,Mia Tharia,\N,\N,"actor,writer,actress",tt8581660
785,nm9919463,Rory Anne Dahl,\N,\N,"actor,actress",tt12483708


In [7]:
#Création de la vue pour joindre Acteur/trics, films, année de sortie du film

sql("""
    CREATE OR REPLACE VIEW act_dec AS (SELECT primaryName
    , knownForTitles
    , original_title
    , YEAR(release_date) AS release_year
    FROM df_act_expl 
    JOIN tmdb ON df_act_expl.knownForTitles = tmdb.imdb_id
    ORDER BY primaryName)
    """
)

,Count


In [8]:
#cCréation de la vue pour ajouter l'intervalle d'années

sql("""
    CREATE OR REPLACE VIEW act_dec_dec AS (SELECT *,
    CASE 
        WHEN release_year BETWEEN 1970 AND 1979 THEN '1970s'
        WHEN release_year BETWEEN 1980 AND 1989 THEN '1980s'
        WHEN release_year BETWEEN 1990 AND 1999 THEN '1990s'
        WHEN release_year BETWEEN 2000 AND 2009 THEN '2000s'
        WHEN release_year BETWEEN 2010 AND 2019 THEN '2010s'
        WHEN release_year BETWEEN 2020 AND 2025 THEN '2020s'
        ELSE 'before 1970'
    END AS decade
    FROM act_dec ORDER BY decade);

    SELECT * FROM act_dec_dec LIMIT 30
    """
)


,primaryName,knownForTitles,original_title,release_year,decade
0,Gloria Lynne Henry,tt0083620,Bare Essence,1982,1980s
1,Gloria Lynne Henry,tt0118971,The Devil's Advocate,1997,1990s
2,Chic,tt0132347,Mystery Men,1999,1990s
3,Gloria Lynne Henry,tt0110823,Phantasm III: Lord of the Dead,1994,1990s
4,Emelyn Bennett,tt0168449,The Adventures of Sebastian Cole,1998,1990s
5,Laurie Sheppard,tt0123189,Neptune's Rocking Horse,1997,1990s
6,Sandra Lee-Oian Thomas,tt0116705,Jingle All the Way,1996,1990s
7,Robin Brenner,tt0337921,Cellular,2004,2000s
8,Brooke Livingston,tt0893367,RoboDoc,2009,2000s
9,Mayuko Kitayama,tt1188701,Gwen Stefani: Harajuku Lovers Live,2006,2000s


In [20]:
df_nbfilm_act = sql(""" 
    SELECT COUNT(knownForTitles) AS nb_films, 
    primaryName,
    decade
    FROM act_dec_dec
    GROUP BY primaryName, decade
    ORDER BY decade, nb_films DESC
    """
)
df_nbfilm_act

,nb_films,primaryName,decade
0,1,Gloria Lynne Henry,1980s
1,2,Gloria Lynne Henry,1990s
2,1,Laurie Sheppard,1990s
3,1,Emelyn Bennett,1990s
4,1,Chic,1990s
...,...,...,...
261,1,Erin Yoo,before 1970
262,1,Julia Belanova,before 1970
263,1,Mia Tharia,before 1970
264,1,Hinano Kuzukawa,before 1970


In [21]:
df_main = sql("""
    SELECT MAX(nb_films) AS nb_film_max,
    primaryName,
    decade
    FROM df_nbfilm_act
    GROUP BY primaryName, decade
    ORDER BY decade, nb_film_max DESC
    """
)
df_main

,nb_film_max,primaryName,decade
0,1,Gloria Lynne Henry,1980s
1,2,Gloria Lynne Henry,1990s
2,1,Sandra Lee-Oian Thomas,1990s
3,1,Emelyn Bennett,1990s
4,1,Chic,1990s
...,...,...,...
261,1,Julia Belanova,before 1970
262,1,Erin Yoo,before 1970
263,1,Mars,before 1970
264,1,Daisy Sequerra,before 1970


In [ ]:
#correction gpt
decades = df_nbfilm_act['decade'].unique()

for decade in decades:
    num_mem = 0
    name_mem = ""

    print(f"Décennie : {decade}")
    
    for idx, row in df_nbfilm_act.iterrows():
        if row['decade'] == decade:
            print(f"  → idx = {idx}, acteur = {row['primaryName']}, films = {row['nb_films']}")
            if row['nb_films'] > num_mem:
                num_mem = row['nb_films']
                name_mem = row['primaryName']
    
    print(f"⭐ Meilleur acteur des {decade} : {name_mem} ({num_mem} films)\n")

Décennie : 1980s
  → idx = 0, acteur = Gloria Lynne Henry, films = 1
⭐ Meilleur acteur des 1980s : Gloria Lynne Henry (1 films)

Décennie : 1990s
  → idx = 1, acteur = Gloria Lynne Henry, films = 2
  → idx = 2, acteur = Laurie Sheppard, films = 1
  → idx = 3, acteur = Emelyn Bennett, films = 1
  → idx = 4, acteur = Chic, films = 1
  → idx = 5, acteur = Sandra Lee-Oian Thomas, films = 1
⭐ Meilleur acteur des 1990s : Gloria Lynne Henry (2 films)

Décennie : 2000s
  → idx = 6, acteur = La Georgea, films = 3
  → idx = 7, acteur = Chic, films = 2
  → idx = 8, acteur = Mayuko Kitayama, films = 2
  → idx = 9, acteur = Natasha Goradia, films = 1
  → idx = 10, acteur = Sarah Finigan, films = 1
  → idx = 11, acteur = ShaynaAngel Monroe, films = 1
  → idx = 12, acteur = Brett Moriarty, films = 1
  → idx = 13, acteur = Pamela Shaw, films = 1
  → idx = 14, acteur = Robin Brenner, films = 1
  → idx = 15, acteur = Bernadette Murray, films = 1
  → idx = 16, acteur = Eve Brand, films = 1
  → idx = 17, 

In [ ]:
#original 
dec = list(df_nbfilm_act['decade'].unique())

num_mem = 0
name_mem = ""

for idx, col in enumerate(df_nbfilm_act['decade']) : 
    print(f"idx ={idx}, col ={col}")
    for i in dec : 
        print(f"i= {i}")
        if col == i :
            num = df_nbfilm_act['nb_films'][idx]
            print(f"nombre film : {num}")
        if num > num_mem :
            name_mem = df_nbfilm_act['primaryName'][idx]
            num_mem = num
    print(name_mem, num_mem, col)


idx =0, col =1980s
i= 1980s
nombre film : 1
i= 1990s
i= 2000s
i= 2010s
i= 2020s
i= before 1970
Gloria Lynne Henry 1 1980s
idx =1, col =1990s
i= 1980s
i= 1990s
nombre film : 2
i= 2000s
i= 2010s
i= 2020s
i= before 1970
Gloria Lynne Henry 2 1990s
idx =2, col =1990s
i= 1980s
i= 1990s
nombre film : 1
i= 2000s
i= 2010s
i= 2020s
i= before 1970
Gloria Lynne Henry 2 1990s
idx =3, col =1990s
i= 1980s
i= 1990s
nombre film : 1
i= 2000s
i= 2010s
i= 2020s
i= before 1970
Gloria Lynne Henry 2 1990s
idx =4, col =1990s
i= 1980s
i= 1990s
nombre film : 1
i= 2000s
i= 2010s
i= 2020s
i= before 1970
Gloria Lynne Henry 2 1990s
idx =5, col =1990s
i= 1980s
i= 1990s
nombre film : 1
i= 2000s
i= 2010s
i= 2020s
i= before 1970
Gloria Lynne Henry 2 1990s
idx =6, col =2000s
i= 1980s
i= 1990s
i= 2000s
nombre film : 3
i= 2010s
i= 2020s
i= before 1970
La Georgea 3 2000s
idx =7, col =2000s
i= 1980s
i= 1990s
i= 2000s
nombre film : 2
i= 2010s
i= 2020s
i= before 1970
La Georgea 3 2000s
idx =8, col =2000s
i= 1980s
i= 1990s
i= 

In [49]:
import plotly.express as px


fig = px.bar(df_nbfilm_act, x='nb_films' ,y= 'primaryName', animation_frame='decade')

fig.update_layout(
    xaxis_range=[0, df_nbfilm_act['nb_films'].max() + 2],
    yaxis_range= [0,20],
    xaxis_title="Nom",
    yaxis_title="Nombre de films",
    showlegend=False
)
fig.show()



In [48]:
import pandas as pd
import plotly.express as px

# Étape 1 — Top 20 acteurs par décennie
df_top20 = (
    df_nbfilm_act
    .sort_values(['decade', 'nb_films'], ascending=[True, False])
    .groupby('decade', group_keys=False)
    .head(20)
)

# Étape 2 — Création de la figure animée
fig = px.bar(
    df_top20,
    x='nb_films',
    y='primaryName',
    animation_frame='decade',
    orientation='h',
    color_discrete_sequence=["#1f77b4"]
)

# Étape 3 — Personnalisation du layout
fig.update_layout(
    xaxis_range=[0, df_top20['nb_films'].max() + 2],
    title="Acteurs les plus prolifiques par décennie",
    xaxis_title="Nombre de films",
    yaxis_title="Nom",
    showlegend=False,
    height=700,
    margin=dict(l=140, r=40, t=60, b=40),
    plot_bgcolor='white',
    font=dict(size=14)
)

# Étape 4 — Tri visuel des barres
fig.update_yaxes(categoryorder="total ascending")

# Étape 5 — Animation plus fluide
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig.layout.sliders[0].currentvalue.prefix = "Décennie : "

# Étape 6 — Affichage
fig.show()

In [240]:
import nbformat
print(nbformat.__version__)

5.10.4


In [10]:
df.head() #tmdb

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0,['Comedy'],None,3924,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,...,70,['en'],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7,['Columbia Pictures'],['US']
1,False,None,0,['Adventure'],None,6124,tt0011436,de,Der Mann ohne Namen,None,...,420,[],Released,None,"Peter Voss, Thief of Millions",False,0.000,0,[],[]
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",None,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,...,110,"['it', 'ja', 'pl', 'fr', 'de']",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.700,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']"
3,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,0,"['Drama', 'Comedy', 'Crime']",None,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,73,['fi'],Released,None,Ariel,False,7.046,248,['Villealfa Filmproductions'],['FI']
4,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,0,"['Drama', 'Comedy', 'Romance']",None,3,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,76,['en'],Released,None,Shadows in Paradise,False,7.182,269,['Villealfa Filmproductions'],['FI']


In [98]:
df_tmdb.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'production_companies_name',
       'production_companies_country'],
      dtype='object')